In [1]:
import numpy as np
from landlab import RasterModelGrid
from landlab.components.overland_flow import OverlandFlow
from landlab.plot.imshow import imshow_grid
from matplotlib.pyplot import figure

First we need to load in the weather data and find the only days where there is rain

In [2]:
import pandas as pd

weather = pd.read_fwf("weather.txt")

# find the days that have positive precipitation

days_with_rain = weather['rain'][weather['rain'] > 0]
days_with_rain = days_with_rain[days_with_rain.index.isin(range(0, 365))]
days_with_rain

0      23.9
3      14.7
4       0.8
9       3.8
15      6.3
       ... 
329     2.0
343     1.3
348    24.9
355     0.5
361     3.3
Name: rain, Length: 106, dtype: float64

In [75]:
days_with_rain.index[1]

3

In [158]:
import netCDF4

class Overland:
    def __init__(self):
        self.shape = (20, 20)
        self.shape_used = (range(2,18), range(2,18))
        self.input = {}
        
    def set_value(self, name, value):
        if name != 'rainfall__depth':
            raise KeyError('"rainfall__depth" is the only valid setter')
        self.input[name] = value
    
    def initialize(self):
        outfile = netCDF4.Dataset('overland.nc', 'w')
        xs, ys = self.shape_used
        outfile.createDimension('x', len(xs))
        outfile.createDimension('y', len(ys))
        self.time_dimension =  outfile.createDimension('time', 365)
#         self.time_dimension.units = 'day'
        self.surface_water__depth = outfile.createVariable('surface_water__depth', 'f4', ('x', 'y', 'time'))
#         self.surface_water__depth.units = 'millimetre'
        self.output = outfile  
    
    def run(self, rainfall_mmhr):
        grid = RasterModelGrid(self.shape)
        topo = np.zeros(self.shape)
        topo[4, 5] = 0.05
        topo[13,3] = 0.1
        grid.at_node['topographic__elevation'] = topo
        grid.at_node['surface_water__depth'] = np.zeros(self.shape)
        
        elapsed_time = 0.0
        model_run_time = 7200.0

        storm_duration = 7200.0
        of = OverlandFlow(grid, steep_slopes=True, rainfall_intensity=rainfall_mmhr * (1/3600 * 1/1000))
        
        while elapsed_time < model_run_time:
            of.dt = of.calc_time_step()     # Adaptive time step

            if elapsed_time > storm_duration:
                of.rainfall_intensity = 0.0

            of.overland_flow()
            elapsed_time += of.dt

        xs, ys = self.shape_used
        
        return grid.at_node['surface_water__depth'].reshape(self.shape)[xs[0]:xs[1], ys[0]:ys[1]]
        
    def update(self):
        rainfall = self.input['rainfall__depth']
        for ind in range(365):
            if ind in rainfall.index:
                grid = self.run(rainfall[ind])
                self.surface_water__depth[:, :, ind] = grid
            else:
                xsize = len(self.shape_used[0])
                ysize = len(self.shape_used[1])
                grid = np.zeros((xsize, ysize))
                self.surface_water__depth[:, :, ind] = grid
                

In [172]:
import os

os.unlink('overland.nc')

of = Overland()
of.set_value('rainfall__depth', days_with_rain)

In [173]:
of.initialize()
of.update()

routing rainfall at t=0
routing rainfall at t=1
routing rainfall at t=2
routing rainfall at t=3
routing rainfall at t=4
routing rainfall at t=5
routing rainfall at t=6
routing rainfall at t=7
routing rainfall at t=8
routing rainfall at t=9
routing rainfall at t=10
routing rainfall at t=11
routing rainfall at t=12
routing rainfall at t=13
routing rainfall at t=14
routing rainfall at t=15
routing rainfall at t=16
routing rainfall at t=17
routing rainfall at t=18
routing rainfall at t=19
routing rainfall at t=20
routing rainfall at t=21
routing rainfall at t=22
routing rainfall at t=23
routing rainfall at t=24
routing rainfall at t=25
routing rainfall at t=26
routing rainfall at t=27
routing rainfall at t=28
routing rainfall at t=29
routing rainfall at t=30
routing rainfall at t=31
routing rainfall at t=32
routing rainfall at t=33
routing rainfall at t=34
routing rainfall at t=35
routing rainfall at t=36
routing rainfall at t=37
routing rainfall at t=38
routing rainfall at t=39
routing ra

In [171]:
data = netCDF4.Dataset('overland.nc', 'r')
data.variables['surface_water__depth'][:,:,3]

masked_array(data =
 [[ 0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726]
 [ 0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726]
 [ 0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726]
 [ 0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726]
 [ 0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008726
   0.0008726  0.0008726  0.0008726  0.0008726]
 [ 0.0008726  0.0008726  0.0008726  0.0008726  0.0008726  0.0008

landlab version 2.2.0